## PROGRAM extrct
Extracts data from the N-body output files Single.dat (fort.83) and Binary.dat (fort.82) relating to the overall cluster evolution, ie. half-mass radius and relaxation time, cluster mass, membership, core radius. 

Note: planets are ignored in this version. 

Output in extrct.dat: 

*   Number of single stars + binaries 
*   Number of binaries 
*   Time (Myr) 
*   Relaxation time (Myr) 
*   Total cluster mass (Msun) 
*   Mass in core (Msun) 
*   Mass outside the tidal radius (Msun) 
*   Maximum stellar distance from cluster centre of mass (pc) 
*   Half-mass radius (pc) 
*   Radius containing inner 10% of cluster mass (pc) 
*   Core radius - as determined by Nbody code (pc) 
*   Number of systems (stars + binaries) inside the half-mass radius   
*   Number of systems within 1pc of the cluster centre    
*   Number of systems within the inner lagrangian radius (10%)  
*   Number of systems within the core radius 
*   Velocity dispersion (km/s)

In [1]:
#Necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

## Reformatting and reading data from fort.83 and fort.82
(Removing format of fort.83 and fort.82 - run this just once)

In [2]:
import os.path

file83 = os.path.isfile('fort.83-awk')
file82 = os.path.isfile('fort.82-awk')

if not file83:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' fort.83 > fort.83-awk
    print "Removing format of fort.83"
if not file82:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22,$23,$24}' fort.82 > fort.82-awk
    print "Removing format of fort.82"

In [21]:
Ns = 0
Nb = 0
with open("fort.83-awk", 'r') as file1:
    with open("fort.82-awk", 'r') as file2:
        while file1 and file2:
            line11 = file1.readline()
            line21 = file2.readline()
            if not (line11 and line21):
                break
            words11 = line11.split()
            words21 = line21.split()
            Ns = int(words11[0])
            Nb = int(words21[0])
            if Ns != -1000:
                line12 = file1.readline()
                words12 = line12.split()
                line13 = file1.readline()
                words13 = line13.split()
                blockS = np.fromfile(file1, sep=' ', count=13*Ns).reshape([Ns,13])
            if Nb != -1000:
                blockB = np.fromfile(file2, sep=' ', count=24*Nb).reshape([Nb,24])
            mtot = sum(blockS[:,2])+sum(blockB[:,8])+sum(blockB[:,9])
            print mtot

423.467
423.467
418.667
418.667
418.53
418.53
418.323
418.323
417.691
417.691
417.408
417.408
417.153
417.153
416.842
416.842
416.352
416.352
405.196
405.196
396.004
396.004
393.886
393.886
392.484
392.484
385.503
385.503
374.435
374.435
374.0
374.0
373.974
373.974
371.262
371.262
369.746
369.746
367.474
367.474
367.441
367.441
365.308
365.308
363.408
363.408
363.373
363.373
360.607
360.607
356.656
356.656
354.346
354.346
335.924
335.924
333.023
333.023
331.698
331.698
312.02
312.02
305.869
305.869
305.76
305.76
305.572
305.572
305.187
305.187
301.555
301.555
291.012
291.012
289.8
289.8
289.384
289.384
286.524
286.524
286.524
286.524
285.443
285.443
283.94
283.94
283.043
283.043
273.994
273.994
269.117
269.117
264.786
264.786
264.393
264.393
261.485
261.485
259.596
259.596
253.67
253.67
249.133
249.133
248.338
248.338
244.363
244.363
242.086
242.086
240.811
240.811
238.792
238.792
237.74
237.74
237.74
237.74
237.74
237.74
237.481
237.481
237.083
237.083
236.615
236.615
235.882
235.882


In [4]:
#Numerical initiation of constants
amin = 1.0e+10
amax = -1.0e+10
vstar = -1
ebcut = 0
trhsum = 0
m0 = 0
rmax = 1
mtot = 0
mout = 0
n1 = 0
n2 = 0
n1pc = 0
vdisp = 0
msgl = 0
nsgl = 0
mwd = 0.0
nwd = 0

In [5]:
#Astronomical constants
pc = 3.0856776e+18
au = pc/1.4959787e+12

In [8]:
#Main call of functions and writing to output file

globaldata(words11,words12,words13,vstar,ebcut)

(387.2089, 1.65, 2.96, 73204429.04385334, -1, 0)

In [7]:
#Global values
def globaldata(words11,words12,words13,vstar,ebcut):
    m0 = float(words13[0])
    rbar = float(words12[3])
    zmbar = float(words13[1])
    su = pc/(au*6.955e+10)*rbar*au
    if vstar == 0:
        vstar = 0.06557*np.sqrt(m0/rbar)
        print ' ZMBAR RBAR VSTAR ',m0,rbar,vstar
        ebcut = (1./zmbar)*(1./zmbar)/(20/su)
    return m0,rbar,zmbar,su,vstar,ebcut

In [ ]:
#Global structural parameters of the stellar cluster
def cstructure():
        rd = np.sqrt((blockS[]-words[])**2+(blockS[]-words[])**2+(blockS[]-words[])**2)

In [67]:
#OPEN(3,file='extrct.dat',status='unknown')